## Úvod

V tejto lekcii sa dozviete:
- Čo je volanie funkcií a na čo sa používa
- Ako vytvoriť volanie funkcie pomocou OpenAI
- Ako integrovať volanie funkcie do aplikácie

## Ciele učenia

Po absolvovaní tejto lekcie budete vedieť a rozumieť:

- Účelu používania volania funkcií
- Nastaviť volanie funkcie pomocou služby OpenAI
- Navrhnúť efektívne volania funkcií pre potreby vašej aplikácie


## Pochopenie volania funkcií

V tejto lekcii chceme vytvoriť funkciu pre náš vzdelávací startup, ktorá umožní používateľom vyhľadávať technické kurzy pomocou chatbota. Budeme odporúčať kurzy, ktoré zodpovedajú ich úrovni zručností, aktuálnej pracovnej pozícii a technológiám, o ktoré majú záujem.

Na realizáciu tohto cieľa použijeme kombináciu:
 - `OpenAI` na vytvorenie chatovacieho zážitku pre používateľa
 - `Microsoft Learn Catalog API` na pomoc používateľom pri vyhľadávaní kurzov podľa ich požiadaviek
 - `Function Calling` na spracovanie používateľskej požiadavky a jej odoslanie do funkcie, ktorá vykoná API požiadavku

Na začiatok sa pozrime, prečo by sme vlastne chceli použiť volanie funkcií:

print("Správy v ďalšej požiadavke:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # získajte novú odpoveď od GPT, kde už vidí odpoveď z funkcie


print(second_response.choices[0].message)


### Prečo používať Function Calling

Ak ste už absolvovali niektorú inú lekciu v tomto kurze, pravdepodobne už chápete silu využívania veľkých jazykových modelov (LLM). Dúfame, že si zároveň uvedomujete aj niektoré ich obmedzenia.

Function Calling je funkcia v službe OpenAI Service, ktorá bola navrhnutá na riešenie týchto problémov:

Nekonzistentné formátovanie odpovedí:
- Pred zavedením function calling boli odpovede z veľkého jazykového modelu neštruktúrované a nekonzistentné. Vývojári museli písať zložitý validačný kód, aby vedeli spracovať každú možnú variáciu výstupu.

Obmedzená integrácia s externými dátami:
- Pred touto funkciou bolo náročné zapojiť údaje z iných častí aplikácie do kontextu chatu.

Štandardizovaním formátu odpovedí a umožnením plynulej integrácie s externými dátami function calling zjednodušuje vývoj a znižuje potrebu ďalšej validačnej logiky.

Používatelia nemohli dostať odpovede ako „Aké je aktuálne počasie v Štokholme?“. Je to preto, že modely boli obmedzené na obdobie, kedy boli trénované na dostupných dátach.

Pozrime sa na príklad nižšie, ktorý ilustruje tento problém:

Povedzme, že chceme vytvoriť databázu údajov o študentoch, aby sme im vedeli odporučiť vhodný kurz. Nižšie máme dva popisy študentov, ktoré sú si veľmi podobné v údajoch, ktoré obsahujú.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

Chceme toto poslať LLM, aby sme spracovali údaje. Neskôr to môžeme použiť v našej aplikácii na odoslanie do API alebo uloženie do databázy.

Vytvorme dva identické prompt-y, ktorými inštruujeme LLM, o aké informácie máme záujem:


Chceme toto poslať LLM, aby analyzoval časti, ktoré sú dôležité pre náš produkt. Takže môžeme vytvoriť dva identické prompti na inštrukciu LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Po vytvorení týchto dvoch výziev ich pošleme do LLM pomocou `openai.ChatCompletion`. Výzvu uložíme do premennej `messages` a priradíme rolu `user`. Toto slúži na napodobnenie správy od používateľa, ktorá je napísaná chatbotovi.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Aj keď sú výzvy rovnaké a popisy podobné, môžeme dostať rôzne formáty vlastnosti `Grades`.

Ak spustíte vyššie uvedenú bunku viackrát, formát môže byť `3.7` alebo `3.7 GPA`.

Je to preto, že LLM prijíma neštruktúrované dáta vo forme napísanej výzvy a tiež vracia neštruktúrované dáta. Potrebujeme mať štruktúrovaný formát, aby sme vedeli, čo očakávať pri ukladaní alebo používaní týchto údajov.

Použitím funkčného volania si môžeme byť istí, že dostaneme späť štruktúrované dáta. Pri použití funkčného volania LLM v skutočnosti nevolá ani nespúšťa žiadne funkcie. Namiesto toho vytvoríme štruktúru, ktorú má LLM pri svojich odpovediach dodržiavať. Potom tieto štruktúrované odpovede použijeme na to, aby sme vedeli, akú funkciu v našich aplikáciách spustiť.


![Diagram toku volania funkcie](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.sk.png)


### Príklady použitia volania funkcií

**Volanie externých nástrojov**  
Chatboti sú skvelí v poskytovaní odpovedí na otázky používateľov. Pomocou volania funkcií môžu chatboti využívať správy od používateľov na vykonanie určitých úloh. Napríklad študent môže požiadať chatbota: „Pošli e-mail môjmu vyučujúcemu, že potrebujem viac pomoci s touto témou.“ Toto môže vyvolať volanie funkcie `send_email(to: string, body: string)`

**Tvorba API alebo databázových dopytov**  
Používatelia môžu vyhľadávať informácie pomocou prirodzeného jazyka, ktorý sa následne premení na formátovaný dopyt alebo API požiadavku. Príkladom môže byť učiteľ, ktorý sa opýta: „Ktorí študenti dokončili poslednú úlohu?“, čo môže vyvolať funkciu s názvom `get_completed(student_name: string, assignment: int, current_status: string)`

**Vytváranie štruktúrovaných údajov**  
Používatelia môžu zobrať blok textu alebo CSV a pomocou LLM z neho vyťažiť dôležité informácie. Napríklad študent môže previesť článok z Wikipédie o mierových dohodách na tvorbu AI kartičiek. Toto sa dá dosiahnuť použitím funkcie `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Vytvorenie prvého volania funkcie

Proces vytvorenia volania funkcie zahŕňa 3 hlavné kroky:
1. Zavolanie Chat Completions API so zoznamom vašich funkcií a správou od používateľa
2. Prečítanie odpovede modelu na vykonanie akcie, napríklad spustenie funkcie alebo API volania
3. Opätovné zavolanie Chat Completions API s odpoveďou z vašej funkcie, aby ste tieto informácie použili na vytvorenie odpovede pre používateľa.


![Tok volania funkcie](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.sk.png)


### Prvky volania funkcie

#### Vstup od používateľa

Prvým krokom je vytvorenie správy od používateľa. Túto správu môžete dynamicky priradiť na základe hodnoty textového vstupu, alebo jej hodnotu môžete zadať priamo tu. Ak s Chat Completions API pracujete prvýkrát, musíme definovať `role` a `content` správy.

`role` môže byť buď `system` (nastavovanie pravidiel), `assistant` (model) alebo `user` (koncový používateľ). Pri volaní funkcie nastavíme túto hodnotu na `user` a zadáme ukážkovú otázku.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Vytváranie funkcií.

Ďalej si definujeme funkciu a jej parametre. V tomto prípade použijeme len jednu funkciu s názvom `search_courses`, ale môžete si vytvoriť aj viacero funkcií.

**Dôležité** : Funkcie sú zahrnuté v systémovej správe pre LLM a budú započítané do počtu dostupných tokenov, ktoré máte k dispozícii.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definície**

Štruktúra definície funkcie má viacero úrovní, pričom každá má svoje vlastné vlastnosti. Tu je rozpis vnorených štruktúr:

**Vlastnosti funkcie na najvyššej úrovni:**

`name` - Názov funkcie, ktorú chceme zavolať.

`description` - Popis toho, ako funkcia funguje. Tu je dôležité byť konkrétny a jasný.

`parameters` - Zoznam hodnôt a formátu, ktoré má model vygenerovať vo svojej odpovedi.

**Vlastnosti objektu Parameters:**

`type` - Dátový typ objektu parameters (zvyčajne "object")

`properties` - Zoznam konkrétnych hodnôt, ktoré model použije vo svojej odpovedi

**Vlastnosti jednotlivých parametrov:**

`name` - Implicitne definované kľúčom vlastnosti (napr. "role", "product", "level")

`type` - Dátový typ tohto konkrétneho parametra (napr. "string", "number", "boolean")

`description` - Popis konkrétneho parametra

**Voliteľné vlastnosti:**

`required` - Pole, ktoré uvádza, ktoré parametre sú povinné na dokončenie volania funkcie


### Volanie funkcie
Po definovaní funkcie ju teraz musíme zahrnúť do volania Chat Completion API. Urobíme to tak, že do požiadavky pridáme `functions`. V tomto prípade `functions=functions`.

Je tu tiež možnosť nastaviť `function_call` na hodnotu `auto`. To znamená, že necháme LLM rozhodnúť, ktorú funkciu má zavolať na základe správy od používateľa, namiesto toho, aby sme to určovali my sami.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Teraz sa pozrime na odpoveď a uvidíme, ako je naformátovaná:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Vidíte, že názov funkcie je volaný a z používateľskej správy dokázal LLM nájsť údaje, ktoré zapadnú do argumentov funkcie.


## 3. Integrácia volaní funkcií do aplikácie.

Po otestovaní formátovanej odpovede z LLM ju môžeme integrovať do aplikácie.

### Riadenie toku

Aby sme to integrovali do našej aplikácie, postupujme podľa týchto krokov:

Najprv zavoláme služby OpenAI a správu uložíme do premennej s názvom `response_message`.


In [ ]:
response_message = response.choices[0].message

Teraz definujeme funkciu, ktorá zavolá Microsoft Learn API na získanie zoznamu kurzov:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Ako najlepšiu prax najskôr zistíme, či model chce zavolať nejakú funkciu. Potom vytvoríme jednu z dostupných funkcií a priradíme ju k tej, ktorú model požaduje.

Následne vezmeme argumenty funkcie a priradíme ich k argumentom z LLM.

Nakoniec pridáme správu o volaní funkcie a hodnoty, ktoré boli vrátené správou `search_courses`. Týmto spôsobom má LLM všetky potrebné informácie na to, aby mohol odpovedať používateľovi prirodzeným jazykom.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Výzva na programovanie

Skvelá práca! Ak chcete pokračovať v učení o OpenAI Function Calling, môžete si vyskúšať: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Pridať ďalšie parametre funkcie, ktoré môžu pomôcť študentom nájsť viac kurzov. Dostupné parametre API nájdete tu:
 - Vytvoriť ďalšie volanie funkcie, ktoré zohľadní viac informácií od študenta, napríklad jeho rodný jazyk
 - Pridať spracovanie chýb, ak volanie funkcie alebo API nevráti žiadne vhodné kurzy



---

**Vyhlásenie o vylúčení zodpovednosti**:  
Tento dokument bol preložený pomocou AI prekladateľskej služby [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, upozorňujeme, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Za autoritatívny zdroj sa považuje pôvodný dokument v jeho natívnom jazyku. Pre kritické informácie odporúčame profesionálny ľudský preklad. Nezodpovedáme za žiadne nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
